## 네이버 뉴스 크롤링 후 빅데이터팀 뉴스채널에 매일 자동 발송

- 로직 
* 네이버 뉴스 피드에서 금융빅데이터로 검색 후 연관성 순으로 10개 정렬
* 관련도 상위 순으로 1건 발송. 단, 이전에 발송된 이력이 있으면 제외한다. 
* 발송 이력은 SQLite로 관리한다. 

In [1]:
import pandas as pd
import numpy as np
import sqlite3

import requests
import json
import os
from bs4 import BeautifulSoup as bs
import time

네이버에서 키워드(금융빅데이터) 관련도 순으로 뉴스를 크롤링한다. 

In [2]:
url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
headers = {'Referer':'https://www.naver.com'
               ,'upgrade-insecure-requests':'1'
               ,'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}

search = '금융빅데이터'    # *** 요거는 임의대로 정하자 *** #
r = requests.get(url.format(search), headers = headers)
soup = bs(r.text, "lxml")

htmls = soup.select("dl > dt > a") 

today_news = []

for html in htmls:
    today_news.append((html.get('href'), html.get('title')))
    
today_news = pd.DataFrame(today_news, columns=['url', 'title'])
today_news = today_news[today_news['title'].notnull()]   # 제목이 없는 url 삭제

stop_words = '빅데이터MSI|시장심리'   # | 구분자로 계속 제외 단어 넣기 
bools = ~today_news['title'].str.contains(stop_words, regex=True)
today_news = today_news[bools]

today_news    

,url,title
1,http://news1.kr/articles/?3653034,"핀테크 등 105곳, 신용정보원 금융빅데이터 이용 신청"
2,http://www.ddaily.co.kr/news/article.html?no=1...,"핀테크에 돈과 인재가 몰린다…금융 혁신, 무한경쟁속으로"
3,http://www.newsis.com/view/?id=NISX20190625_00...,"[빅데이터MSI]시장심리 톱5, LG전자·삼성전자·KB금융·네이버·LG생활건강"
4,http://www.hankookilbo.com/news/npath/20190624...,"“IT공룡들, 금융시장 독점할 수 있다”… 국제결제은행의 경고"
5,http://www.dhnews.co.kr/news/articleView.html?...,"글로벌 융합형 금융 비즈니스 인재 양성, 한국외대 국제금융학과"
6,http://www.dtoday.co.kr/news/articleView.html?...,"빅3 ETF 운용사, 각사별 경쟁력 살펴보니"
7,http://www.meconomynews.com/news/articleView.h...,[데이터+] '공인중개사 20만명 응시' 기사에... 화나요·슬퍼요 92%
8,http://kor.theasian.asia/archives/225457,"[인터뷰] 홍성욱 과학커뮤니케이터 “빅데이터가 낳는 '빅브라더' 우려"""
9,http://www.kbanker.co.kr/news/articleView.html...,[금융을 살리자] (4) 코너 몰린 카드사 '디지털화'로 승부수
10,http://www.fntimes.com/html/view.php?ud=201906...,[금융보안원 김영기 원장] 데이터 혁명으로 가는 길 ‘금융보안'


기존에 발송된 뉴스 리스트를 내린다.  
만약 처음으로 작업하는 경우, 현재 크롤링한 데이터를 기존 데이터로 밀어넣는다. 

In [3]:
con = sqlite3.connect("news_list.db")

try: 
    all_news = pd.read_sql("select * from all_news", con)
except:
    print("데이터가 없네요. 현재 데이터를 넣습니다.")    
    today_news.to_sql('all_news', con, if_exists='append', index=False)  # index=False 인덱스는 넣지 않는다. 
    all_news = pd.read_sql("select * from all_news", con)
    
pd.read_sql("select * from all_news", con)    

,url,title
0,http://kor.theasian.asia/archives/225457,"[인터뷰] 홍성욱 과학커뮤니케이터 “빅데이터가 낳는 '빅브라더' 우려"""
1,http://www.fntimes.com/html/view.php?ud=201906...,[금융보안원 김영기 원장] 데이터 혁명으로 가는 길 ‘금융보안'
2,http://www.fnnews.com/news/201906231243246914,빅데이터로 국가 경쟁력 제고…조직 맞춤형 정책 필요
3,http://www.businesspost.co.kr/BP?command=artic...,"더존비즈온 비즈니스온 웹케시, 기업 빅데이터 소유해 몸값 올라"
4,http://www.econovill.com/news/articleView.html...,"빅데이터, 류현진부터 스타벅스까지"
5,http://www.businesspost.co.kr/BP?command=artic...,"류영준, 새 운동장 마이데이터산업에서도 카카오페이 주도권 쥘까"
6,http://www.weeklytoday.com/news/articleView.ht...,[빅데이터 금융] 빅데이터 금융 확장위해 '탈피' 시급
7,https://www.hankyung.com/economy/article/20190...,"스타트업 키우는 은행들…""빅·보·상에 꽂혔다"""
8,http://www.kyeonggi.com/news/articleView.html?...,"인천시, 네이버 데이터센터 유치 검토...유치 효과 놓고 찬반 팽팽"
9,http://www.newspim.com/news/view/20190621000115,"[리포트 브리핑]NICE, 'Corp. Day 후기: 빅데이터 사업 기회와 자회사 ..."


In [68]:
# [참고] SQL 원본 테이블을 지우고 싶다면, 
#con.cursor().execute("DROP TABLE IF EXISTS all_news")

크롤링 뉴스 중 관련도 최상위 1건 발송! 

* 크롤링된 뉴스를 순차적으로 기존 발송분과 중복 체크한다. 
* 중복시 pass
* 겹치지 않으면 슬랙 발송하고, SQLite에 업데이트 후 break!

In [4]:
def check_dup_url(u, ser):
    return ser.apply(lambda x: True if u == x else False).any()  # 한건이라도 겹치면, True 반환

def send_slack(msg):
    webhook_url = "YOUR_URL_ADDR"
    content = msg
    payload = {"text": content}
    requests.post(
        webhook_url, data=json.dumps(payload),
        headers={'Content-Type': 'application/json'})
    
for i, row in today_news.iterrows():
    #print(ser)
    if check_dup_url(row.url, all_news['url']):   # 중복 있으면
        pass
    else:
        #print(i, row.url, row.title)
        msg = (row.title + '\n' + row.url)  # 제목+URL로 구성
        #print(msg)
        send_slack(msg)
        df_add = pd.DataFrame({'url':[row.url], 'title':[row.title]})
        df_add.to_sql('all_news', con, if_exists='append', index=False)
        break    